# imports

In [1]:
import sketch
import time 

In [2]:
import pandas as pd  
import numpy as np

import time 
import re
import os 
import json

In [3]:
# webdriver: provides the API for accessing and controlling a web browser.
from selenium import webdriver

# Keys: provides keys that can be used for keyboard interactions.
from selenium.webdriver.common.keys import Keys

# By: provides mechanisms to locate elements on a web page.
from selenium.webdriver.common.by import By

# WebDriverWait: provides a way to wait for a specific condition to occur before proceeding with the code execution.
from selenium.webdriver.support.ui import WebDriverWait

# expected_conditions: provides a set of predefined conditions that can be used with WebDriverWait to wait for an element to become available, clickable, or visible.
from selenium.webdriver.support import expected_conditions as EC

In [4]:
from bs4 import BeautifulSoup

# Acquire Selenium

In [4]:
season_list = [1,2,2018,2019,2020,2021]

# Season 1 / 2015

In [5]:
# Create an instance of the Chrome browser
driver = webdriver.Chrome()

# Navigate to the BattleBots website
driver.get("https://battlebots.com/season-1-robots/")

# Get the page source
html = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, "html.parser")


In [6]:
# Find all the <div> elements with class="cbp-wrapper"
div_elements = soup.find_all("div", {"class": "cbp-wrapper"})



In [7]:
links = []
# Find all the <div> elements with class="cbp-wrapper"
div_elements = soup.find_all("div", {"class": "cbp-wrapper"})

# Iterate over the list of matching <div> elements
for div in div_elements:
    # Find all the <div> elements with class="more-button has-love" within the current <div> element
    more_buttons = div.find_all("div", {"class": "more-button has-love"})
    
    # For example, you could extract the href attribute of the <a> element within each <div>
    for button in more_buttons:
        link = button.find("a")["href"]
        links.append(link)


# JSON
    

In [21]:
df

Stats,robot,Total matches,Win percentage,Total wins,Losses,Knockouts,KO percentage,Average knockout time,Knockouts against,KO against percentage,Judges decision wins
0,Beta,4,75%,3,1,1,25%,180s,0,0%,2


In [5]:
# open .json using open
with open('all_season_robot_links.json') as json_file:
    data = json.load(json_file)

In [6]:
# checking response
data

{'world-championship-vii-robots': ['https://battlebots.com/robot/banshee-wcvii/',
  'https://battlebots.com/robot/beta-wcvii/',
  'https://battlebots.com/robot/big-dill-wcvii/',
  'https://battlebots.com/robot/black-dragon-wcvii/',
  'https://battlebots.com/robot/blip-wcvii/',
  'https://battlebots.com/robot/bloodsport-wcvii/',
  'https://battlebots.com/robot/captain-shredderator-wcvii/',
  'https://battlebots.com/robot/claw-viper-wcvii/',
  'https://battlebots.com/robot/cobalt-wcvii/',
  'https://battlebots.com/robot/copperhead-wcvii/',
  'https://battlebots.com/robot/deathroll-wcvii/',
  'https://battlebots.com/robot/doom-wcvii/',
  'https://battlebots.com/robot/doomba-wcvii/',
  'https://battlebots.com/robot/double-tap-wcvii/',
  'https://battlebots.com/robot/dragon-king-wcvii/',
  'https://battlebots.com/robot/emulsifier-wcvii/',
  'https://battlebots.com/robot/end-game-wcvii/',
  'https://battlebots.com/robot/free-shipping-wcvii/',
  'https://battlebots.com/robot/fusion-wcvii/',
 

In [205]:
# looking at key names
data.keys()

dict_keys(['world-championship-vii-robots', '2021-season-robots', '2020-season-robots', '2019-season-robots', '2018-season-robots', 'season-2-robots', 'season-1-robots'])

In [7]:
# testing response
data['world-championship-vii-robots'][0]

'https://battlebots.com/robot/banshee-wcvii/'

In [343]:
# create an empty DataFrame to store concatenated tables
all_tables = pd.DataFrame()

# for every key in data do this
for key in data.keys():
    
    # show when new key is accessed
    print(key)
    
    # for every item in key, do this.
    for i in data[key]:
        
        # show current link being scraped
        print(i)
        
        # try and continue
        try:
            
            # read tables
            tables = pd.read_html(i)
        
        except ValueError:
        # No tables found, skip to the next link
            continue
        
        try:
            # take table 0 and store it in table
            table = tables[0]
            
            # filter by data frame
            table = table[['Stats','Career']].T
        
        
            # set the first row as column names
            table.columns = table.iloc[0]
            
            # drop unhelpful index
            table = table.drop(index='Stats')
        
            # reset the index
            table = table.reset_index(drop= True)
            
            # renaming the link(i) to url
            url = f'{i}'
            
            # search through url for match
            match = re.search(r'/([^/]+)/?$', url)
            
            # if match then assign name
            if match:
            
                table['robot'] = match.group(1)
                
        except KeyError:
            continue
            
        # add current table to the all_tables DataFrame
        all_tables = pd.concat([all_tables, table], ignore_index=True,axis = 0)
        print(all_tables.shape)
        time.sleep(1)

world-championship-vii-robots
https://battlebots.com/robot/banshee-wcvii/
(1, 11)
https://battlebots.com/robot/beta-wcvii/
(2, 11)
https://battlebots.com/robot/big-dill-wcvii/
(3, 11)
https://battlebots.com/robot/black-dragon-wcvii/
(4, 11)
https://battlebots.com/robot/blip-wcvii/
(5, 11)
https://battlebots.com/robot/bloodsport-wcvii/
(6, 11)
https://battlebots.com/robot/captain-shredderator-wcvii/
(7, 11)
https://battlebots.com/robot/claw-viper-wcvii/
(8, 11)
https://battlebots.com/robot/cobalt-wcvii/
(9, 11)
https://battlebots.com/robot/copperhead-wcvii/
(10, 11)
https://battlebots.com/robot/deathroll-wcvii/
(11, 11)
https://battlebots.com/robot/doom-wcvii/
(12, 11)
https://battlebots.com/robot/doomba-wcvii/
(13, 11)
https://battlebots.com/robot/double-tap-wcvii/
(14, 11)
https://battlebots.com/robot/dragon-king-wcvii/
(15, 11)
https://battlebots.com/robot/emulsifier-wcvii/
(16, 11)
https://battlebots.com/robot/end-game-wcvii/
(17, 11)
https://battlebots.com/robot/free-shipping-wcvii

(151, 11)
https://battlebots.com/robot/kraken-2020/
(152, 11)
https://battlebots.com/robot/lock-jaw-2020/
(153, 11)
https://battlebots.com/robot/madcatter-2020/
(154, 11)
https://battlebots.com/robot/malice-2020/
(155, 11)
https://battlebots.com/robot/mammoth-2020/
(156, 11)
https://battlebots.com/robot/p1-2020/
(157, 11)
https://battlebots.com/robot/pain-train-2020/
(158, 11)
https://battlebots.com/robot/perfect-phoenix-2020/
(159, 11)
https://battlebots.com/robot/rampage-2020/
(160, 11)
https://battlebots.com/robot/ribbot-2020/
(161, 11)
https://battlebots.com/robot/rotator-2020/
(162, 11)
https://battlebots.com/robot/rusty-2020/
(163, 11)
https://battlebots.com/robot/sawblaze-2020/
(164, 11)
https://battlebots.com/robot/sharko-2020/
(165, 11)
https://battlebots.com/robot/shatter-2020/
(166, 11)
https://battlebots.com/robot/skorpios-2020/
(167, 11)
https://battlebots.com/robot/slammow-2020/
(168, 11)
https://battlebots.com/robot/slap-box-2020/
(169, 11)
https://battlebots.com/robot/s

(301, 11)
https://battlebots.com/robot/witch-doctor-2018/
(302, 11)
https://battlebots.com/robot/yeti-2/
(303, 11)
season-2-robots
https://battlebots.com/robot/ambush-s2/
(304, 11)
https://battlebots.com/robot/bad-kitty/
(305, 11)
https://battlebots.com/robot/basilisk/
(306, 11)
https://battlebots.com/robot/beta-2/
(307, 11)
https://battlebots.com/robot/bite-force-2/
(308, 11)
https://battlebots.com/robot/black-ice/
(309, 11)
https://battlebots.com/robot/blacksmith/
(310, 11)
https://battlebots.com/robot/bombshell/
(311, 11)
https://battlebots.com/robot/bronco-2/
(312, 11)
https://battlebots.com/robot/brutus/
(313, 11)
https://battlebots.com/robot/bucktooth-burl-s2/
(314, 11)
https://battlebots.com/robot/captain-shrederator-2/
(315, 11)
https://battlebots.com/robot/chomp-2/
(316, 11)
https://battlebots.com/robot/chromefly-s2/
(317, 11)
https://battlebots.com/robot/cobalt-s2/
(318, 11)
https://battlebots.com/robot/complete-control-2/
(319, 11)
https://battlebots.com/robot/creepy-crawlie

In [345]:
all_tables.to_csv('all-table1.csv')

# Create A Function for Reproducibility

# tabel 1

In [359]:


def stats_df():
    """
    A function that reads robot statistics from a list of links to web pages,
    filters and formats them to fit into a Pandas DataFrame, and concatenates all the resulting
    DataFrames into one master DataFrame.
    This uses a file named 'all_season_robot_links.json'

    Returns:
    --------
    Pandas DataFrame:
        A DataFrame containing robot statistics from all the links in the provided JSON file.
    """
    
    # read in the JSON file containing the robot web page links
    with open('all_season_robot_links.json') as json_file:
        data = json.load(json_file)

    # create an empty DataFrame to store concatenated tables
    all_tables = pd.DataFrame()

    # loop through every key in data
    for key in data.keys():

        print(f"Scraping statistics for {key} robots...")

        # loop through every link associated with the current key
        for i in data[key]:

            print(f"\tScraping {i}...")

            # try to read tables from the current link
            try:
                tables = pd.read_html(i)

            # if no tables are found, skip to the next link
            except ValueError:
                continue

            # try to format the first table in the resulting list of tables
            try:
                # select and transpose the relevant rows from the table
                table = tables[0][['Stats', 'Career']].T

                # use the first row as column names
                table.columns = table.iloc[0]

                # drop the now redundant row of column names
                table = table.drop(index='Stats')

                # reset the row index to start from 0
                table = table.reset_index(drop=True)

                # extract the robot name from the current link and add it as a column to the table
                url = f'{i}'
                match = re.search(r'/([^/]+)/?$', url)
                if match:
                    table['robot'] = match.group(1)

            # if any errors occur while formatting the table, skip to the next link
            except KeyError:
                continue

            # add the current table to the master DataFrame
            all_tables = pd.concat([all_tables, table], ignore_index=True, axis=0)
            print(f"\t\t...table added to master DataFrame ({all_tables.shape[0]} rows)")

            # pause briefly to avoid overloading the web server with requests
            time.sleep(1)

    print("...done!")
    # return the completed master DataFrame
    # save to csv
    all_tables.to_csv('all-table1.csv'')
    return all_tables

In [9]:
stats_df()

# table 2


In [35]:
# create an empty DataFrame to store concatenated tables
all_tables = pd.DataFrame()

# for every key in data do this
for key in data.keys():
    
    # show when new key is accessed
    print(key)
    
    # for every item in key, do this.
    for i in data[key]:
        
        # show current link being scraped
        print(i)
        
        # try and continue
        try:
            
            # read tables
            tables = pd.read_html(i)
        
        except ValueError:
            # No tables found, skip to the next link
            continue
        
        try:
            tables = pd.read_html(i)
            if len(tables) < 2:
                continue
            table = tables[1]
            # rest of the code
            
            
            # extract the robot name from the current link and add it as a column to the table
            url = f'{i}'
            match = re.search(r'/([^/]+)/?$', url)
            if match:
                table['robot'] = match.group(1)

        except KeyError:
            continue
            
        # add current table to the all_tables DataFrame
        all_tables = pd.concat([all_tables, table], ignore_index=True,axis=0)
        print(all_tables.shape)
        time.sleep(1)

world-championship-vii-robots
https://battlebots.com/robot/banshee-wcvii/
(4, 5)
https://battlebots.com/robot/beta-wcvii/
(17, 5)
https://battlebots.com/robot/big-dill-wcvii/
(29, 5)
https://battlebots.com/robot/black-dragon-wcvii/
(55, 5)
https://battlebots.com/robot/blip-wcvii/
(66, 5)
https://battlebots.com/robot/bloodsport-wcvii/
(93, 5)
https://battlebots.com/robot/captain-shredderator-wcvii/
(118, 5)
https://battlebots.com/robot/claw-viper-wcvii/
(131, 5)
https://battlebots.com/robot/cobalt-wcvii/
(149, 5)
https://battlebots.com/robot/copperhead-wcvii/
(169, 5)
https://battlebots.com/robot/deathroll-wcvii/
(182, 5)
https://battlebots.com/robot/doom-wcvii/
(182, 5)
https://battlebots.com/robot/doomba-wcvii/
(184, 5)
https://battlebots.com/robot/double-tap-wcvii/
(185, 5)
https://battlebots.com/robot/dragon-king-wcvii/
(185, 5)
https://battlebots.com/robot/emulsifier-wcvii/
(189, 5)
https://battlebots.com/robot/end-game-wcvii/
(221, 5)
https://battlebots.com/robot/free-shipping-wcv

(2511, 5)
https://battlebots.com/robot/jackpot-2020/
(2525, 5)
https://battlebots.com/robot/kraken-2020/
(2552, 5)
https://battlebots.com/robot/lock-jaw-2020/
(2587, 5)
https://battlebots.com/robot/madcatter-2020/
(2609, 5)
https://battlebots.com/robot/malice-2020/
(2626, 5)
https://battlebots.com/robot/mammoth-2020/
(2649, 5)
https://battlebots.com/robot/p1-2020/
(2663, 5)
https://battlebots.com/robot/pain-train-2020/
(2670, 5)
https://battlebots.com/robot/perfect-phoenix-2020/
(2677, 5)
https://battlebots.com/robot/rampage-2020/
(2682, 5)
https://battlebots.com/robot/ribbot-2020/
(2707, 5)
https://battlebots.com/robot/rotator-2020/
(2738, 5)
https://battlebots.com/robot/rusty-2020/
(2745, 5)
https://battlebots.com/robot/sawblaze-2020/
(2776, 5)
https://battlebots.com/robot/sharko-2020/
(2784, 5)
https://battlebots.com/robot/shatter-2020/
(2804, 5)
https://battlebots.com/robot/skorpios-2020/
(2835, 5)
https://battlebots.com/robot/slammow-2020/
(2844, 5)
https://battlebots.com/robot/sl

(4862, 5)
https://battlebots.com/robot/whiplash-s3/
(4897, 5)
https://battlebots.com/robot/witch-doctor-2018/
(4935, 5)
https://battlebots.com/robot/yeti-2/
(4955, 5)
season-2-robots
https://battlebots.com/robot/ambush-s2/
https://battlebots.com/robot/bad-kitty/
(4956, 6)
https://battlebots.com/robot/basilisk/
(4957, 6)
https://battlebots.com/robot/beta-2/
(4961, 6)
https://battlebots.com/robot/bite-force-2/
(4970, 6)
https://battlebots.com/robot/black-ice/
(4972, 6)
https://battlebots.com/robot/blacksmith/
(4975, 6)
https://battlebots.com/robot/bombshell/
(4981, 6)
https://battlebots.com/robot/bronco-2/
(4990, 6)
https://battlebots.com/robot/brutus/
(4993, 6)
https://battlebots.com/robot/bucktooth-burl-s2/
(4994, 6)
https://battlebots.com/robot/captain-shrederator-2/
(4998, 6)
https://battlebots.com/robot/chomp-2/
(5005, 6)
https://battlebots.com/robot/chromefly-s2/
(5007, 6)
https://battlebots.com/robot/cobalt-s2/
(5009, 6)
https://battlebots.com/robot/complete-control-2/
(5012, 6)
h

In [38]:
all_tables.to_csv('match.csv')